# Postprocessing 

This notebook provides code to delete buffer created around SVF tiles and merge them back into one tile. 

In [33]:
import rasterio
from rasterio.merge import merge
import glob
import os

os.chdir('/Users/artyom/Documents/GitHub/HeatExposureModel')
print(os.getcwd())

/Users/artyom/Documents/GitHub/HeatExposureModel


In [36]:
# Get list of all raster files
# raster_files = glob.glob('data/Trees_exp_SVF/*.TIF')
raster_files = glob.glob('data/Buildings_exp_SVF/*.TIF')
raster_files.sort()
raster_files

['data/Buildings_exp_SVF/svf_expanded_Buildings_0.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_1.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_2.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_3.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_4.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_5.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_6.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_7.TIF',
 'data/Buildings_exp_SVF/svf_expanded_Buildings_8.TIF']

In [37]:
# Function to remove buffer and prepare for merging
def remove_buffer(src, buffer_size=40):
    # Read the raster
    with rasterio.open(src) as dataset:
        # Get the data excluding buffer
        data = dataset.read(1)[buffer_size:-buffer_size, buffer_size:-buffer_size]
        
        # Get the transform
        transform = dataset.transform
        # Adjust transform to account for removed buffer
        new_transform = rasterio.Affine(transform.a, transform.b, 
                                      transform.c + (buffer_size * transform.a),
                                      transform.d, transform.e, 
                                      transform.f + (buffer_size * transform.e))
        
        return data, new_transform, dataset.meta

In [39]:
# Process and merge files
src_files_to_mosaic = []
for raster in raster_files:
    data, transform, meta = remove_buffer(raster)
    
    # Update metadata
    meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": transform
    })
    
    # Create temporary file with buffered data
    temp_path = f"temp_{os.path.basename(raster)}"
    with rasterio.open(temp_path, 'w', **meta) as dst:
        dst.write(data, 1)
    
    # Add to list for merging
    src_files_to_mosaic.append(rasterio.open(temp_path))

# Merge all files
mosaic, out_trans = merge(src_files_to_mosaic)

# Write final merged file
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans
})

with rasterio.open("data/final/building_svf.tif", "w", **out_meta) as dest:
    dest.write(mosaic)

# Clean up
for src in src_files_to_mosaic:
    src.close()
    os.remove(src.name)